<a href="https://colab.research.google.com/github/HARASEON/Deep_Learning_tensorflow/blob/main/Day5_sunspot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sunspot
1) Lambda : Lambda 함수는 단순히 예측값에 400을 곱하므로 val_loss가 커짐
  * why? 인공신경망에 keras가 제공하지 않는 레이어를 새로 만들기 위해서
  * ex. 차원확장. 스케일업 등
  
2) No lambda
* 데이터셋 특징:
  * Training data: 3000ea
  * Validation data = 235ea

# Time series data
* 예시 : 공장/건강/운동/환자 센서데이터, 주식, 날씨(강우량, 온도, 습도,풍속, 기압 등),기업매출액/매출품목/주문량/생산량/고객만족도, 기업 분기/연차보고서
* 시계열 데이터의 정답은 어떻게 만들어지는가?
  * 데이터를 묶어서(window) 바로 다음 데이터를 정답(label)으로 삼고 다음 데이터의 예측구간(horizon)을 예측한다.
* 사용 예시: 
  * 시계열 데이터들 법칙에서 따라서 다음 생성되는 데이터를 예측
  * 수요예측, 강우량예측, 온도예측
  * 자동차 센서데이터에 의한 기계고장 예측
  * 엔진 예지정비
* 전처리: window + horizon, shuffling = k-fold cross validation
  * why? 데이터를 model이 모두 외우기 때문에 이를 방지하기 위해서

In [5]:
!pip uninstall tensorflow
!pip install tensorflow==2.10.0

Found existing installation: tensorflow 2.10.0
Uninstalling tensorflow-2.10.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.9/dist-packages/tensorflow-2.10.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.10.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow-2.10.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (578.1 MB)


# Sunspot-Lambda
* 손코딩
* target val_loss: 13.3
* 마지막 dense에 왜 1이 들어가고 relu가 안들어가는지 궁금해서 아래를 비교해보았다.
  * Dense(1),
  * Dense(6, relu) -> val_loss가 위보다 더 낮게 나왔다.

In [9]:
import csv
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.layers import Dense, LSTM, Lambda, Conv1D, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.models import Sequential 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import Huber

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  series = tf.expand_dims(series, axis = -1)
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window_size + 1, shift = 1, drop_remainder = True)
  ds = ds.flat_map(lambda w: w.batch(window_size + 1))
  ds = ds.shuffle(shuffle_buffer)
  ds = ds.map(lambda w: (w[:-1], w[1:]))
  return ds.batch(batch_size).prefetch(1)

def solution_model():
  url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
  urllib.request.urlretrieve(url, 'sunspots.csv')

  time_step = []
  sunspots = []

  with open('sunspots.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter = ',')
    next(reader)
    for row in reader:
      sunspots.append(float(row[2]))
      time_step.append(int(row[0]))

    series = np.array(sunspots)
    time = np.array(time_step)

    split_time = 3000

    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]

    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000

    tf.keras.backend.clear_session()
    tf.random.set_seed(53)
    np.random.seed(53)
    train_set  = windowed_dataset(x_train, window_size = window_size, batch_size = batch_size, shuffle_buffer = shuffle_buffer_size)

    validation_set= windowed_dataset(x_valid, window_size = window_size, batch_size = batch_size, shuffle_buffer = shuffle_buffer_size)

    model = Sequential([
        Conv1D(filters=16, kernel_size=5,
               padding = 'causal',
               activation = 'relu',
               input_shape =[None, 1]),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(64, return_sequences=True)),
        Dropout(0.5),
        Dense(64, activation = "relu"),
        Dense(32, activation = "relu"),
        Dense(1),
        Lambda(lambda x :x*400)
    ])
    
    optimizer= SGD(learning_rate = 1e-4, momentum = 0.9)
    loss = Huber()
    model.compile(loss = loss,
                  optimizer = optimizer,
                  metrics =["mae"])
    checkpoint_path = 'checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path,
      save_weights_only = True,
      save_best_only = True, 
      monitor = 'val_mae',
      verbose = 1)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10)
    epochs = 150
    history = model.fit(train_set,
                        validation_data = (validation_set),
                        epochs = epochs,
                        callbacks =[checkpoint, early_stopping])
    model.load_weights(checkpoint_path)
    return model


if __name__ == '__main__':
  model = solution_model()
  model.save("sunspot_lambda.h5")

  from google.colab import files
  files.download('sunspot_lambda.h5')  

Epoch 1/150
     92/Unknown - 9s 15ms/step - loss: 38.4115 - mae: 38.9076
Epoch 1: val_mae improved from inf to 17.12928, saving model to checkpoint.ckpt
93/93 [==============================] - 10s 34ms/step - loss: 38.2428 - mae: 38.7389 - val_loss: 16.6357 - val_mae: 17.1293
Epoch 2/150
92/93 [============================>.] - ETA: 0s - loss: 20.9220 - mae: 21.4158
Epoch 2: val_mae improved from 17.12928 to 12.33761, saving model to checkpoint.ckpt
93/93 [==============================] - 2s 17ms/step - loss: 20.9060 - mae: 21.3997 - val_loss: 11.8449 - val_mae: 12.3376
Epoch 3/150
93/93 [==============================] - ETA: 0s - loss: 17.4893 - mae: 17.9819
Epoch 3: val_mae did not improve from 12.33761
93/93 [==============================] - 2s 16ms/step - loss: 17.4893 - mae: 17.9819 - val_loss: 12.5631 - val_mae: 13.0536
Epoch 4/150
91/93 [============================>.] - ETA: 0s - loss: 15.7663 - mae: 16.2580
Epoch 4: val_mae improved from 12.33761 to 10.08084, saving model

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ChatGPT
* prompt:
code with the dataset from 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv' .  use Lambda(lambda predicted_value :predicted_value*400), RNN and transfer learning to get val_loss under 13.3

* responds:
pretrain 모델을 사용하려 해서 실패패

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load the data
url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
sunspots_df = pd.read_csv(url, parse_dates=['Date'], index_col='Date')

# Prepare the data for training and testing
train_size = int(len(sunspots_df) * 0.8)
train_df = sunspots_df.iloc[:train_size]
test_df = sunspots_df.iloc[train_size:]

# Normalize the data
train_mean = train_df.mean()
train_std = train_df.std()
train_df = (train_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

# Define a function to create sequences from the data
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i])
        y.append(data[i])
    X = np.array(X)
    y = np.array(y)
    return X, y

# Define the sequence length
seq_length = 30

# Create the training and testing sequences
train_X, train_y = create_sequences(train_df.values, seq_length)
test_X, test_y = create_sequences(test_df.values, seq_length)

# Load a pre-trained model
base_model = keras.models.load_model('/path/to/pretrained/model.h5')
base_model.trainable = False

# Add a Lambda layer to the model
model = keras.Sequential([
    base_model,
    layers.Lambda(lambda predicted_value: predicted_value * 400)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
)

# Train the model
history = model.fit(
    train_X, train_y,
    epochs=100,
    batch_size=32,
    validation_data=(test_X, test_y),
    verbose=1,
)

# Evaluate the model
val_loss = history.history['val_loss'][-1]
print(f"Validation loss: {val_loss:.4f}")


OSError: ignored

# pretrained model 없이 다시 시도

* loss : mse를 사용한 것을 볼 수 있다.
  * huber loss를 썼던 위 코드보다 mse가 sunspot dataset에는 val_loss가 10 이상 낮게 나왔다.
* callback함수를 쓰라는 명령을 내려서 뒷부분을 수정했다.



In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, LSTM, Lambda, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the data
url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
sunspots_df = pd.read_csv(url, parse_dates=['Date'], index_col='Date')

# Prepare the data for training and testing 
# train : test = 8:2
train_size = int(len(sunspots_df) * 0.8)
train_df = sunspots_df.iloc[:train_size]
test_df = sunspots_df.iloc[train_size:]

# Normalize the data
train_mean = train_df.mean()
train_std = train_df.std()
train_df = (train_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

# Define a function to create sequences from the data
def create_sequences(data, seq_length):
    X = []
    y = []

    for i in range(len(data) - seq_length):
        # Extract the sequence of length seq_length
        seq = data[i : i + seq_length + 1]

        # Split the sequence into features (input) and target (output)
        X.append(seq[:-1, 1]) # select only the second column for each row to create the sequence
        y.append(seq[-1, 1])

    # Convert the lists to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # Reshape the input data to have shape (num_samples, seq_length, input_dim)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, y


# Define the sequence length
seq_length = 30

# Create the training and testing sequences
train_X, train_y = create_sequences(train_df.values, seq_length)
test_X, test_y = create_sequences(test_df.values, seq_length)

# Define the RNN model
model = keras.Sequential([
    layers.LSTM(64, input_shape=(seq_length, 1)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1),
    layers.Lambda(lambda x: x * 400)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
)

# Define the callbacks
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10)
checkpoint = ModelCheckpoint('best_model.h5', monitor = 'val_loss',save_best_only = True)
checkpoint_path = 'checkpoint.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
    save_weights_only=True, 
    save_best_only=True, 
    monitor='val_loss', 
    verbose=1)

# Train the model
history = model.fit(
    train_X, train_y,
    epochs=100,
    batch_size=32,
    validation_data=(test_X, test_y),
    callbacks= [early_stopping, checkpoint],
    verbose=1,
)

model.summary()
model.load_weights(checkpoint_path)
model.save('sunspot_lambda.h5')
# Evaluate the model
val_loss = history.history['val_loss'][-1]
print(f"Validation loss: {val_loss:.4f}")


Epoch 1/100
73/80 [==========================>...] - ETA: 0s - loss: 40.4002
Epoch 1: val_loss improved from inf to 1.15531, saving model to checkpoint.ckpt
80/80 [==============================] - 4s 16ms/step - loss: 37.0060 - val_loss: 1.1553
Epoch 2/100
80/80 [==============================] - ETA: 0s - loss: 0.7452
Epoch 2: val_loss improved from 1.15531 to 0.45906, saving model to checkpoint.ckpt
80/80 [==============================] - 1s 9ms/step - loss: 0.7452 - val_loss: 0.4591
Epoch 3/100
72/80 [==========================>...] - ETA: 0s - loss: 0.4732
Epoch 3: val_loss improved from 0.45906 to 0.36965, saving model to checkpoint.ckpt
80/80 [==============================] - 1s 7ms/step - loss: 0.4747 - val_loss: 0.3697
Epoch 4/100
73/80 [==========================>...] - ETA: 0s - loss: 0.3408
Epoch 4: val_loss improved from 0.36965 to 0.28212, saving model to checkpoint.ckpt
80/80 [==============================] - 1s 7ms/step - loss: 0.3465 - val_loss: 0.2821
Epoch 5/100
7

# Sunspot - NO Lambda
* 손코딩
* val_loss : < 0.07 success! 0.03
* 궁금한것 : 왜 huber loss를 사용하는 데 metric에 mae를 사용해야만 하나요? 에러메시지는 그냥 matricfunction에 없어서 그렇다고 하는데 만들면 비효율적이어서 안만든 것인가요? mae가 다 설명할 수 있어서 굳이 만들지 않은 건가요?



In [23]:
# QUESTION
#
# For this task you will need to train a neural network
# to predict sunspot activity using the Sunspots.csv dataset.
# Your neural network must  have an MAE
# of 0.12 or less on the normalized dataset for top marks.
# Code for normalizing the data is provided and should not be changed.
# At the bottom of this file, we provide  some testing
# code should you want to check your model.

# Note: Do not use lambda layers in your model, they are not supported
# on the grading infrastructure.
import csv
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.layers import Dense, LSTM, Lambda, Conv1D, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import Huber

# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
    urllib.request.urlretrieve(url, 'sunspots.csv')

    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            sunspots.append(float(row[2])) # YOUR CODE HERE)
            time_step.append(int(row[0]))  # YOUR CODE HERE)

    series = np.array(sunspots) # YOUR CODE HERE
    time = np.array(time_step)

    # DO NOT CHANGE THIS CODE
    # This is the normalization function
    min = np.min(series)
    max = np.max(series)
    series -= min
    series /= max
    time = np.array(time_step)

    # The data should be split into training and validation sets at time step 3000
    # DO NOT CHANGE THIS CODE
    split_time = 3000


    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]

    # DO NOT CHANGE THIS CODE
    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000

    tf.keras.backend.clear_session()
    tf.random.set_seed(53)
    np.random.seed(53)

    train_set = windowed_dataset(x_train, 
        window_size=window_size, 
        batch_size=batch_size, 
        shuffle_buffer=shuffle_buffer_size)
    
    validation_set = windowed_dataset(x_valid, 
        window_size=window_size,
        batch_size=batch_size,
        shuffle_buffer=shuffle_buffer_size)

    model = Sequential([
        tf.keras.layers.Conv1D(filters=60, kernel_size=5,
        padding="causal",
        activation="relu",
        input_shape=[None, 1]),
        Bidirectional(LSTM(32, return_sequences=True)),
        Bidirectional(LSTM(32, return_sequences=True)),
        Bidirectional(LSTM(32, return_sequences=True)),
        Dropout(0.5),
        Dense(64, activation="relu"),     
    
                                              
      # YOUR CODE HERE. Whatever your first layer is, the input shape will be [None,1] when using the Windowed_dataset above, depending on the layer type chosen
      tf.keras.layers.Dense(1, activation = "relu")
    ])
    # PLEASE NOTE IF YOU SEE THIS TEXT WHILE TRAINING -- IT IS SAFE TO IGNORE
    # BaseCollectiveExecutor::StartAbort Out of range: End of sequence
    # 	 [[{{node IteratorGetNext}}]]
    #


    # YOUR CODE HERE TO COMPILE AND TRAIN THE MODEL
 #   optimizer = SGD(learning_rate=1e-4, momentum=0.9)
    loss= Huber()
    model.compile(loss=loss,
        optimizer="adam",
        metrics=["mae"]) # "huber"가 안되는 이유?
    checkpoint_path = 'checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path, 
        save_weights_only=True, 
        save_best_only=True, 
        monitor='val_mae',
        verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    epochs=100
    history = model.fit(train_set, 
        validation_data=(validation_set), 
        epochs=epochs, 
        callbacks=[checkpoint, early_stopping],
        )
    model.load_weights(checkpoint_path)
    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, this .h5 model will be
# sent to the testing infrastructure for scoring.

# You must use the Submit and Test button to submit your model
# at least once in each category before you finally submit your exam.

if __name__ == '__main__':
    model = solution_model()
    model.save("sunspot_nolambda.h5")

# 시험볼때는 이줄 아래는 복사하지 마세요
    from google.colab import files
    files.download('sunspot_nolambda.h5')



Epoch 1/100
     93/Unknown - 14s 20ms/step - loss: 0.0076 - mae: 0.0914
Epoch 1: val_mae improved from inf to 0.04614, saving model to checkpoint.ckpt
93/93 [==============================] - 16s 45ms/step - loss: 0.0076 - mae: 0.0914 - val_loss: 0.0021 - val_mae: 0.0461
Epoch 2/100
91/93 [============================>.] - ETA: 0s - loss: 0.0034 - mae: 0.0606
Epoch 2: val_mae improved from 0.04614 to 0.04318, saving model to checkpoint.ckpt
93/93 [==============================] - 2s 23ms/step - loss: 0.0034 - mae: 0.0605 - val_loss: 0.0019 - val_mae: 0.0432
Epoch 3/100
91/93 [============================>.] - ETA: 0s - loss: 0.0028 - mae: 0.0541
Epoch 3: val_mae improved from 0.04318 to 0.04046, saving model to checkpoint.ckpt
93/93 [==============================] - 3s 31ms/step - loss: 0.0028 - mae: 0.0543 - val_loss: 0.0018 - val_mae: 0.0405
Epoch 4/100
92/93 [============================>.] - ETA: 0s - loss: 0.0029 - mae: 0.0557
Epoch 4: val_mae improved from 0.04046 to 0.03985, 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ChatGPT
* val_loss < 0.07 fail 0.11 -> 모델이 dense 하나에 LSTM하나다.
* prompt: code RNN and transfer model with the dataset from url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv' and use the callback, checkpoint,and EarlyStopping. save the best model into '.h5' file  to get lower val_loss under 0.07 without any error. 
* response: 
1. Loading and preprocessing the data: The code loads the sunspot dataset from a URL using pandas, and then preprocesses the data by normalizing it. Normalizing the data helps to ensure that each feature is on a similar scale, which can improve the performance of the model.

2. Defining the model architecture: The code defines a simple Recurrent Neural Network (RNN) model using the Keras API from TensorFlow 2.x.  ** The model consists of a single LSTM layer with 32 units and a dense output layer with 1 unit.**  The input sequences have a window size of 12, and the output is a single prediction for the next time step.

3. Creating and training the model: The code creates training and validation datasets using a sliding window approach, where each input sequence consists of 12 time steps and the corresponding output is the next time step. The model is then trained using the fit method, with early stopping and checkpoint callbacks to save the best model based on the validation loss. The model is trained for a maximum of 50 epochs.

4. Evaluating the best model: After training the model, the code loads the best model saved by the checkpoint callback and evaluates it on the validation dataset. The validation loss is used to measure the performance of the model.

5. Using callbacks to improve model performance: The code uses early stopping and checkpoint callbacks to improve the performance of the model. Early stopping stops training if the validation loss does not improve for a specified number of epochs, which can help to prevent overfitting. Checkpointing saves the best model based on the validation loss, which can be useful if training is interrupted or if the best model needs to be loaded for later use.

In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the sunspot dataset
url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
df = pd.read_csv(url, parse_dates=['Date'], index_col='Date')
series = df['Monthly Mean Total Sunspot Number'].values.reshape(-1, 1)

# Normalize the data
data_mean = series.mean(axis=0)
data_std = series.std(axis=0)
series = (series - data_mean) / data_std

# Define the window size for input sequences and output predictions
window_size = 12
batch_size = 32
shuffle_buffer_size = 1000

# Define a function to create input/output sequences from the time series data
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

# Create the training and validation datasets
train_size = int(len(series) * 0.8)
train_series = series[:train_size]
valid_series = series[train_size:]
train_dataset = windowed_dataset(train_series, window_size, batch_size, shuffle_buffer_size)
valid_dataset = windowed_dataset(valid_series, window_size, batch_size, shuffle_buffer_size)

# Define the model architecture
inputs = Input(shape=(window_size, 1))
x = LSTM(32, activation='relu')(inputs)
outputs = Dense(1)(x)
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(train_dataset, epochs=50, validation_data=valid_dataset, callbacks=[early_stopping, checkpoint])

# Evaluate the best model on the validation set
best_model = tf.keras.models.load_model('best_model.h5')
val_loss = best_model.evaluate(valid_dataset)
print('Validation loss of best model:', val_loss)


Epoch 1/50
81/81 [==============================] - 5s 28ms/step - loss: 0.3981 - val_loss: 0.1766
Epoch 2/50
81/81 [==============================] - 2s 25ms/step - loss: 0.1776 - val_loss: 0.1295
Epoch 3/50
81/81 [==============================] - 2s 27ms/step - loss: 0.1518 - val_loss: 0.1223
Epoch 4/50
81/81 [==============================] - 3s 43ms/step - loss: 0.1498 - val_loss: 0.1235
Epoch 5/50
81/81 [==============================] - 2s 25ms/step - loss: 0.1511 - val_loss: 0.1216
Epoch 6/50
81/81 [==============================] - 2s 26ms/step - loss: 0.1438 - val_loss: 0.1190
Epoch 7/50
81/81 [==============================] - 2s 26ms/step - loss: 0.1460 - val_loss: 0.1198
Epoch 8/50
81/81 [==============================] - 3s 42ms/step - loss: 0.1458 - val_loss: 0.1182
Epoch 9/50
81/81 [==============================] - 2s 26ms/step - loss: 0.1454 - val_loss: 0.1183
Epoch 10/50
81/81 [==============================] - 2s 25ms/step - loss: 0.1463 - val_loss: 0.1273
Epoch 11/

20/20 [==============================] - 0s 9ms/step - loss: 0.1159
Validation loss of best model: 0.11587566137313843


# 재시도
* prompt:  modify this code to get val_loss under 0.07 without any error.
* responds : 
1. Increased the number of epochs to 100: In order to achieve a lower validation loss, the number of epochs has been increased from 50 to 100. This will allow the model more time to learn and converge to a better solution.

2. Increased the patience of EarlyStopping to 10: The patience parameter of EarlyStopping has been increased from 5 to 10. This means that the training will stop only if the validation loss does not improve for 10 consecutive epochs.

3. Reduced the learning rate of the optimizer: The learning rate of the optimizer has been reduced by setting the learning_rate parameter of the Adam optimizer to a smaller value. This can help the optimizer converge to a better solution.

4. Added Dropout layer to prevent overfitting: In order to prevent overfitting, a Dropout layer has been added after the LSTM layer.

5. Changed the Dense layer activation function to 'linear'

In [30]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the sunspot dataset
url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
df = pd.read_csv(url, parse_dates=['Date'], index_col='Date')
series = df['Monthly Mean Total Sunspot Number'].values.reshape(-1, 1)

# Normalize the data
data_mean = series.mean(axis=0)
data_std = series.std(axis=0)
series = (series - data_mean) / data_std

# Define the window size for input sequences and output predictions
window_size = 12
batch_size = 32
shuffle_buffer_size = 1000

# Define a function to create input/output sequences from the time series data
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

# Create the training and validation datasets
train_size = int(len(series) * 0.8)
train_series = series[:train_size]
valid_series = series[train_size:]
train_dataset = windowed_dataset(train_series, window_size, batch_size, shuffle_buffer_size)
valid_dataset = windowed_dataset(valid_series, window_size, batch_size, shuffle_buffer_size)

# Define the model architecture
inputs = Input(shape=(window_size, 1))
x = LSTM(32, activation='relu')(inputs)
x = LSTM(32, activation='relu')(inputs)
x = Dense(32, activation='relu')(x)
outputs = Dense(1)(x)
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(train_dataset, epochs=100, validation_data=valid_dataset, callbacks=[early_stopping, checkpoint])

# Evaluate the best model on the validation set
best_model = tf.keras.models.load_model('best_model.h5')
val_loss = best_model.evaluate(valid_dataset)
print('Validation loss of best model:', val_loss)


Epoch 1/100
81/81 [==============================] - 6s 45ms/step - loss: 0.3887 - val_loss: 0.1469
Epoch 2/100
81/81 [==============================] - 2s 26ms/step - loss: 0.1582 - val_loss: 0.1416
Epoch 3/100
81/81 [==============================] - 2s 26ms/step - loss: 0.1511 - val_loss: 0.1205
Epoch 4/100
81/81 [==============================] - 2s 25ms/step - loss: 0.1480 - val_loss: 0.1239
Epoch 5/100
81/81 [==============================] - 3s 37ms/step - loss: 0.1459 - val_loss: 0.1198
Epoch 6/100
81/81 [==============================] - 2s 25ms/step - loss: 0.1462 - val_loss: 0.1199
Epoch 7/100
81/81 [==============================] - 2s 25ms/step - loss: 0.1440 - val_loss: 0.1191
Epoch 8/100
81/81 [==============================] - 2s 25ms/step - loss: 0.1441 - val_loss: 0.1198
Epoch 9/100
81/81 [==============================] - 3s 40ms/step - loss: 0.1429 - val_loss: 0.1176
Epoch 10/100
81/81 [==============================] - 2s 26ms/step - loss: 0.1426 - val_loss: 0.1173

20/20 [==============================] - 1s 9ms/step - loss: 0.1150
Validation loss of best model: 0.11498843133449554
